In [25]:
# load config for api key
import os
import json
from google import genai
from google.genai import types
import pandas as pd
from tqdm import tqdm
import numpy as np
from PIL import Image

In [26]:
config_path = f"/Users/dhirendrachoudhary/Desktop/Workstation/Research/APIGenie/data/config.json"

if os.path.exists(config_path):
    with open(config_path, 'r') as file:
        config = json.load(file)


api_key = config['llm_api']['api_key']
model_name = config['llm_api']['model_name']

client = genai.Client(api_key=api_key)

In [27]:
image_captions = pd.read_csv('image_captions.csv')
sdxl_dataset_path = 'sdxl_outputs'
sd_2_dataset_path = 'sd_2_outputs'
sdxl_mask_path = os.path.join(sdxl_dataset_path, 'masks')
sd_2_mask_path = os.path.join(sd_2_dataset_path, 'masks')

In [28]:
def get_centroid(mask):
    mask_array = np.array(mask)
    mask_coordinates = np.column_stack(np.where(mask_array > 0))
    if len(mask_coordinates) == 0:
        centroid = (None, None)
    else:
        # Calculate the median of the coordinates
        centroid = np.mean(mask_coordinates, axis=0)
        centroid = centroid[1], centroid[0]
    return tuple(centroid)

In [29]:
for image_name in tqdm(os.listdir(sdxl_dataset_path)):
    if not image_name.endswith('jpg'):
        continue
    idx = int(image_name.split('.')[0])
    image_path = os.path.join(sdxl_dataset_path, image_name)
    img = Image.open(image_path)
    mask_dir = [i for i in os.listdir(sdxl_mask_path) if idx==int(i.split('-')[0])][0]
    mask_dir = os.path.join(sdxl_mask_path, mask_dir)
    dense_captions = []
    for root, nouns, masks in os.walk(mask_dir):
        for mask in masks:
            mask_path = os.path.join(root, mask)
            mask = Image.open(mask_path)
            centroid = get_centroid(mask)
            


  0%|          | 0/201 [00:00<?, ?it/s]


IndexError: list index out of range

In [ ]:
prompt = """I am providing you with captions for sub-regions of an image. These captions will be provided by the corresponding centroids
for the objects in the sub-regions. I want you to stitch all the dense captions into one unified caption for the entire image.
You have to use the centroid information to deduce the relative positions of each of the objects. Do not add any new information
to the captions. Make the caption as short as possible without losing too many details. Any mention of a black background should be ignored. Do not hallucinate any details. Generate the final caption within the <caption></caption> tags."""

response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction="You are a meta image captioning model. You look at various sub-captions and create a meaningful grounded caption using those. You can  use additional provided information to facilitate spatial reasoning."),
    contents="Hello there"
)

'gemini-1.5-flash'